In [37]:
import numpy as np
np.set_printoptions(suppress = True)
import torch
import argparse

# import models
# from utils import calculate_stats

# conda install conda-forge::argparse

In [3]:
# Paula's calculate_stats function
def calculate_stats(X_train, y_train, X_test, y_test, args):

    # Overlapping columns are first 24 for y_train and last 24 for X_train
    means = np.mean(y_test[:, :24] + X_test[:, 8:], axis = 0)

    X_log_eps = log_transform(X_train)
    y_log_eps = log_transform(y_train)
    
    # so4 has much higher values
    # why are these 4 and 5 not 7?
    # These are 17, what are the others?
    so4 = np.concatenate((X_train[:,8], X_train[:,9], X_train[:,10], X_train[:,11], X_train[:,12]), axis = 0)
    bc = np.concatenate((X_train[:,13], X_train[:,14], X_train[:,15], X_train[:,16]), axis = 0)
    oc = np.concatenate((X_train[:,17], X_train[:,18], X_train[:,19], X_train[:,20]), axis = 0)
    # ORDER?!
    du = np.concatenate((X_train[:,21], X_train[:,24], X_train[:,22], X_train[:,23]), axis = 0)

    so4_mean = np.mean(so4)
    bc_mean = np.mean(bc)
    oc_mean = np.mean(oc)  
    du_mean = np.mean(du) 
    
    stats = {'xtrain_mean': np.mean(X_train, axis = 0),
            'xtrain_std': np.std(X_train, axis = 0),
            'ytrain_mean': np.mean(y_train, axis = 0), # this is now delta's
            'ytrain_std': np.std(y_train, axis = 0),
            'so4_mean': so4_mean,
            'bc_mean': bc_mean,
            'oc_mean': oc_mean,
            'du_mean': du_mean,
            'X_log_eps_mean': np.mean(X_log_eps, axis = 0),
            'X_log_eps_std': np.std(X_log_eps, axis = 0),
            'y_log_eps_mean': np.mean(y_log_eps, axis = 0),
            'y_log_eps_std': np.std(y_log_eps, axis = 0),
            'means': means # These are for test
            }
    
    global mu_y
    global si_y
    global mu_x
    global si_x
    
    mu_y = Variable(torch.Tensor(stats['ytrain_mean']), requires_grad = True).to(device)
    si_y = Variable(torch.Tensor(stats['ytrain_std']), requires_grad = True).to(device)
    mu_x = Variable(torch.Tensor(stats['xtrain_mean']), requires_grad = True).to(device)
    si_x = Variable(torch.Tensor(stats['xtrain_std']), requires_grad = True).to(device)
    return stats

In [25]:
# define full path 
path_to_data = "/home/kim/data/aerosols/aerosol_emulation_data/"

X_test = np.load(path_to_data + 'X_test.npy')
y_test = np.load(path_to_data + 'y_test.npy')

X_train = np.load(path_to_data + 'X_train.npy')
y_train = np.load(path_to_data + 'y_train.npy')

# Select the correct 24 columns
X_test_24 = X_test[:, 8:]
X_train_24 = X_train[:, 8:] 

y_test_24 = y_test[:, :24]
y_train_24 = y_train[:, :24]

# How much has it changes between x (at t = 0)  and y (at t = 1)
y_delta_train_24 = y_train_24 - X_train_24
y_delta_test_24 = y_test_24 - X_test_24

In [51]:
i = 1000
print(X_train_24[i, [0, 1, 2, 3, 4]])
print(y_train_24[i, [0, 1, 2, 3, 4]])
print(y_train_24[i, [0, 1, 2, 3, 4]] - X_train_24[i, [0, 1, 2, 3, 4]])

[    34.6611889    1508.99534109 306207.73492199 401794.78072114
     61.04572957]
[    34.65558446   1508.9734313  306207.68081635 401794.86232588
     61.04574469]
[-0.00560444 -0.02190978 -0.05410565  0.08160475  0.00001512]


In [62]:
transition_matrix = np.array([
    [0.9, 0.025, 0.05, 0.025, 0.0],
    [0.025, 0.9, 0.05, 0.025, 0.0],
    [0.0, 0.025, 0.9, 0.025, 0.05],
    [0.0, 0.025, 0.025, 0.9, 0.05],
    [0.0, 0.025, 0.05, 0.025, 0.9]])

result = transition_matrix @ X_train_24[i, [0, 1, 2, 3, 4]].reshape(-1, 1)

print(X_train_24[i, [0, 1, 2, 3, 4]].sum())
print(result.sum())

709607.2179026894
732570.1984326714


In [93]:
tm = torch.tensor([
    [0.9, 0.025, 0.05, 0.025, 0.0],
    [0.025, 0.9, 0.05, 0.025, 0.0],
    [0.0, 0.025, 0.9, 0.025, 0.05],
    [0.0, 0.025, 0.025, 0.9, 0.05],
    [0.05, 0.025, 0.0, 0.025, 0.9]])

print(tm.sum(dim = 1))
print(tm.sum(dim = 0))

tensor([1.00, 1.00, 1.00, 1.00, 1.00])
tensor([0.97, 1.00, 1.02, 1.00, 1.00])


In [96]:
torch.set_printoptions(precision = 2, sci_mode = False)
inp = torch.tensor(X_train_24[i, [0, 1, 2, 3, 4]], dtype = torch.float32)

print(inp)
print(inp.sum())
print(inp @ tm)
print((inp @ tm).sum())
print(tm @ inp)
print((tm @ inp).sum())

tensor([   34.66,  1509.00, 306207.75, 401794.78,    61.05])
tensor(709607.25)
tensor([   71.97, 19060.55, 285709.03, 369310.62, 35455.07])
tensor(709607.25)
tensor([ 25424.18,  26714.22, 285672.62, 369311.25,  10139.27])
tensor(717261.50)


In [80]:
X_train_24[i, [0, 1, 2, 3, 4]].sum()

709607.2179026894

In [ ]:
# Check
so4_indices = [0, 1, 2, 3, 4]
bc_indices = [5, 6, 7, 8]
oc_indices = [9, 10, 11, 12]
du_indices = [13, 14, 15, 16] 
# consider order swap as in summary
# du_indices = [13, 16, 14, 15] 

# X_train_24[:, [0, 1, 2, 3, 4]].sum(axis = 1) - y_train_24[:, [0, 1, 2, 3, 4]].sum(axis = 1)
X_train_24_proportions = X_train_24[:, [0, 1, 2, 3, 4]] / X_train_24[:, [0, 1, 2, 3, 4]].sum(axis = 1)[..., np.newaxis]

array([[ 0.00002747,  0.00157305,  0.41735254,  0.58086694,  0.00017999],
       [ 0.00002747,  0.00157305,  0.41735254,  0.58086694,  0.00017999],
       [ 0.00002747,  0.00157305,  0.41735254,  0.58086694,  0.00017999],
       ...,
       [-0.        ,  0.00004815,  0.97397783,  0.02546609,  0.00050793],
       [-0.        ,  0.00004819,  0.97366943,  0.02576394,  0.00051844],
       [ 0.        ,  0.00004828,  0.97254076,  0.02685941,  0.00055154]])

In [ ]:
# [:, 0] first value is 1000 to 106209.9375, increases monotonically
# [:, 1] second value is 180 to 268
# [:, 2] third column is 0 to 1 (many 1s)
# [:, 3] fourth column is 0.95 to 40.5 (many 1s)
# [:, 4] fifth column is 0 to 1 (mostly 0s, many 1s)
# [:, 5] sixth column is 0 or 1 (manly 0s), nothing inbetween
# [:, 6] seventh column is 0 to 1 (many 0s)
# [:, 7] eighth column is very small values

array([7.00000000e+03, 2.12533646e+02, 7.18839243e-02, 1.83690460e+01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 2.15525649e-21,
       1.29102745e+05, 2.98289010e+01, 5.85955621e+06, 3.67345229e+08,
       1.20037845e+06, 4.62011533e-05, 1.30665184e-03, 2.09419851e-07,
       7.53481217e-09, 1.51886712e-04, 6.19863535e-03, 6.48942320e-07,
       2.67016086e-08, 3.80929956e-03, 2.17798216e-03, 2.91863057e-07,
       5.05673839e-06, 1.43374012e+00, 1.41705490e+01, 1.71655660e+01,
       9.95631738e-03, 2.82922001e-05, 1.83177483e-06, 6.29868841e-06])

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# within args the default mode is train
def main(args):
    path_to_data = "/home/kim/data/aerosols/aerosol_emulation_data/"
    
    print('Loading data')    
    X_train = np.load(path_to_data + 'X_train.npy')
    y_train = np.load(path_to_data + 'y_train.npy')  
    
    if args.mode == 'train':
    # during training select validation as test data
        X_test = np.load(path_to_data + 'X_val.npy')
        y_test = np.load(path_to_data + 'y_val.npy')
    else:
        X_test = np.load(path_to_data + 'X_test.npy')
        y_test = np.load(path_to_data + 'y_test.npy')
    

    # Calculate tendencies (i.e. deltas)
    # THIS IS AN OVERWRITE
    # After this the y are deltas, not absolute values
    y_train[:, :24] -= X_train[:, 8:]
    y_test[:, :24] -= X_test[:, 8:]

    # Calculate stats
    # stats = calculate_stats(X_train, y_train, X_test, y_test, args)